<a href="https://colab.research.google.com/github/karinstefa/test_colab/blob/main/lab1_karinmunoz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import gensim
from gensim.parsing.porter import PorterStemmer 
from gensim.parsing.preprocessing import remove_stopwords
from gensim import corpora
from gensim import models
from gensim import similarities
import nltk

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

#nltk.download()


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: ignored

In [ ]:
#%%
# 1. Load the data
lyrics_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/lab1/input/lyrics-data.csv')
artists_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/lab1/input/artists-data.csv')


In [ ]:
# %%
lyrics_data_en = lyrics_data[lyrics_data['language'] == 'en']

In [ ]:
lyrics_data_en['textos_lyrics'] = lyrics_data_en.SName+' '+lyrics_data_en.Lyric

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
# 2. Preprocess the data
#remover espacios dobles y triples
lyrics_data_en['textos_lyrics'] = lyrics_data_en['textos_lyrics'].str.replace('\n', ' ')
lyrics_data_en['textos_lyrics'] = lyrics_data_en['textos_lyrics'].str.replace('[^a-zA-Z]|[0-9]', ' ')
lyrics_data_en['textos_lyrics'] = lyrics_data_en['textos_lyrics'].str.replace('\s+', ' ')


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row

In [ ]:
lyrics_data_en = lyrics_data_en.dropna(subset= ['textos_lyrics'])

In [ ]:
lyrics_data_en['textos_lyrics_clean'] = lyrics_data_en.textos_lyrics.apply(remove_stopwords)

In [ ]:
p = PorterStemmer()

lyrics_data_en['textos_lyrics_clean'] = lyrics_data_en.textos_lyrics_clean.apply(p.stem_sentence)

In [ ]:
def preprocessing_data(data):
    data_nor = str(data).lower()
    return nltk.word_tokenize(data_nor)


In [ ]:
lyrics_data_en['textos_lyrics_clean'] = lyrics_data_en['textos_lyrics_clean'].apply(preprocessing_data)

In [ ]:
docDict = lyrics_data_en['textos_lyrics_clean'].to_list()

In [ ]:
dictionary_text = corpora.Dictionary(docDict)
dictionary_text.save('/content/drive/MyDrive/Colab Notebooks/lab1/output/dictionary.dict')
print(dictionary_text)
print(dictionary_text.token2id)

Dictionary(142845 unique tokens: ['and', 'as', 'better', 'but', 'call']...)
{'and': 0, 'as': 1, 'better': 2, 'but': 3, 'call': 4, 'careless': 5, 'chanc': 6, 'cheat': 7, 'comfort': 8, 'crowd': 9, 'd': 10, 'danc': 11, 'di': 12, 'easi': 13, 'ey': 14, 'feel': 15, 'feet': 16, 'floor': 17, 'fool': 18, 'forev': 19, 'friend': 20, 'given': 21, 'gon': 22, 'gone': 23, 'good': 24, 'goodby': 25, 'got': 26, 'guilti': 27, 'hand': 28, 'heart': 29, 'hurt': 30, 'i': 31, 'ignor': 32, 'kind': 33, 'know': 34, 'known': 35, 'lead': 36, 'leav': 37, 'live': 38, 'll': 39, 'lose': 40, 'loud': 41, 'love': 42, 'm': 43, 'mayb': 44, 'mend': 45, 'mind': 46, 'music': 47, 'na': 48, 'never': 49, 'now': 50, 'pain': 51, 'pleas': 52, 'pretend': 53, 'rhythm': 54, 's': 55, 'sad': 56, 'screen': 57, 'should': 58, 'silver': 59, 'so': 60, 'stai': 61, 'that': 62, 'the': 63, 'there': 64, 'thing': 65, 'though': 66, 'time': 67, 'to': 68, 'tonight': 69, 'truth': 70, 'unsur': 71, 've': 72, 'wai': 73, 'want': 74, 'wast': 75, 'we': 76, 

In [ ]:
class MyCorpus(object):
    def __iter__(self):
        for line in docDict:
            yield dictionary_text.doc2bow(line)

corpus_memory_friendly = MyCorpus()
corpora.MmCorpus.serialize('/content/drive/MyDrive/Colab Notebooks/lab1/output/corpus.mm', corpus_memory_friendly)

In [ ]:
corpus = corpora.MmCorpus('/content/drive/MyDrive/Colab Notebooks/lab1/output/corpus.mm')

In [ ]:
dictonary = corpora.Dictionary.load('/content/drive/MyDrive/Colab Notebooks/lab1/output/dictionary.dict')
corpus = corpora.MmCorpus('/content/drive/MyDrive/Colab Notebooks/lab1/output/corpus.mm')
Tfidf = models.TfidfModel(corpus)

In [ ]:
query = 'in the end'
query_doc_bow = dictonary.doc2bow(preprocessing_data(query))
print(query_doc_bow)
print(Tfidf[query_doc_bow])

[(31, 1), (42, 1), (123, 1)]
[(31, 0.14045590189777965), (42, 0.8019068500497674), (123, 0.5807043511679081)]


In [ ]:
index_doc = similarities.MatrixSimilarity(Tfidf[corpus])
index_doc.save('/content/drive/MyDrive/Colab Notebooks/lab1/output/index_doc.index')

In [ ]:
index_doc = similarities.MatrixSimilarity.load('/content/drive/MyDrive/Colab Notebooks/lab1/output/index_doc.index')
sims = index_doc[Tfidf[query_doc_bow]]
print(list(enumerate(sims)))